# Original Jeepney Routes - Fitness Function

## Route Connection and Genetic Algorithm Functions

### Setting Up

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import shapely
import folium
import geojson
import math
import osmnx as ox
from rtree import index as rtree_index
import pickle
import copy
import utm

from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from geopy.distance import geodesic
from shapely.ops import split

from __future__ import absolute_import, division
from math import radians, sin, cos, sqrt, atan2, exp, log
import webbrowser
import random
from scipy.spatial import KDTree
from scipy.spatial.distance import euclidean

ox.settings.log_console=True
ox.settings.use_cache=True

In [ ]:
# Defining classes for the dataframes      
class stopCandidate:
    def __init__(self, lat, long, isTranspo, id):
        self.lat = lat
        self.long = long
        self.isTranspo = isTranspo
        self.id = id #Stop ID
        
    def getLat(self):
        return self.lat
    
    def getLong(self):
        return self.long
    
    def getDegree(self):
        return self.degree
    
class networkObj:
    def __init__(self, route_network, graph):
        self.fitness_score = 0
        self.graph = graph
        self.route_network = route_network
        
class Route:
    def __init__(self, route, route_id, distance):
        self.route = route
        self.route_id = route_id
        self.distance = distance

In [ ]:
# For units
def degrees_to_meters(angle_degrees):
    return angle_degrees * 6371000 * math.pi / 180

def meters_to_degrees(distance_meters):
    return distance_meters / 6371000 * 180 / math.pi

In [ ]:
# Import and Export networks or graphs to pickle
def export_networks(networks, path):
    with open(path, 'wb') as f:
        pickle.dump(networks, f)

def import_networks(path):
    with open(path, 'rb') as f:
        routes = pickle.load(f)
    return routes

#### Graph and Features

In [ ]:
# GENERATION OF MAIN CITY GRAPH
# IF FIRST TIME RUNNING, RUN THIS CODE TO GENERATE THE GRAPH
def generate_graph():
    mode = 'drive'
    graph = ox.graph_from_place(select_city, network_type = mode) # Generate graph of Metro manila
    ox.save_graphml(graph, city_file) # Save it as a file

def load_graph():
    graph = ox.load_graphml(city_file)
    
    print("Graph loaded successfully")
    print("NUMBER OF EDGES: ", graph.number_of_edges())
    print("NUMBER OF NODES: ", graph.number_of_nodes())
    print('\n')
    return graph

select_city = "Metro Manila, Philippines"
city_file = 'map/Metro Manila.graphml'
#generate_graph()
METROMANILA_GRAPH = load_graph()

select_city = "Manila, Philippines"
city_file = 'map/Manila.graphml'
#generate_graph()
MANILA_GRAPH = load_graph()

select_city = "Mandaluyong, Philippines"
city_file = 'map/Mandaluyong.graphml'
#generate_graph()
MANDALUYONG_GRAPH = load_graph()

select_city = "Makati, Philippines"
city_file = 'map/Makati.graphml'
#generate_graph()
MAKATI_GRAPH = load_graph()

In [ ]:
### For Filtering the roads and other features
# GETTING ROADS AND WATERWAYS

# Get all the roads in Manila
manila_road = ox.graph_to_gdfs(MANILA_GRAPH,nodes=False, edges=True)
filtered_roads = manila_road[manila_road['junction'].isna()]

# Separate roads whose highway types are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

# Allowed Roads to place stops
filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']

# Get all the roads with the allowed road types
filtered_roads_strings_manila = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 

# Total Distance of the allowed road types
TOTAL_NETWORK_ROAD_DISTANCE_MANILA = 0
for index, row in filtered_roads_strings_manila.iterrows():
    TOTAL_NETWORK_ROAD_DISTANCE_MANILA += row['length']
    
# MAKATI
makati_road = ox.graph_to_gdfs(MAKATI_GRAPH,nodes=False, edges=True)
filtered_roads = makati_road[makati_road['junction'].isna()]

# Separate roads whose highway types are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

# Allowed Roads to place stops
filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']

# Get all the roads with the allowed road types
filtered_roads_strings_makati = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 

# Total Distance of the allowed road types
TOTAL_NETWORK_ROAD_DISTANCE_MAKATI = 0
for index, row in filtered_roads_strings_makati.iterrows():
    TOTAL_NETWORK_ROAD_DISTANCE_MAKATI += row['length']


# MANDALUYONG
mandaluyong_road = ox.graph_to_gdfs(MANDALUYONG_GRAPH,nodes=False, edges=True)
filtered_roads = mandaluyong_road[mandaluyong_road['junction'].isna()]

# Separate roads whose highway types are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

# Allowed Roads to place stops
filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']

# Get all the roads with the allowed road types
filtered_roads_strings_mandaluyong = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 

# Total Distance of the allowed road types
TOTAL_NETWORK_ROAD_DISTANCE_MANDALUYONG = 0
for index, row in filtered_roads_strings_mandaluyong.iterrows():
    TOTAL_NETWORK_ROAD_DISTANCE_MANDALUYONG += row['length']

### Reading Data

In [ ]:
# Reading all points

manila_points_gdf = gpd.read_file('./City Data/Manila_point.geojson')
mandaluyong_points_gdf = gpd.read_file('./City Data/Mandaluyong_point.geojson')
makati_points_gdf = gpd.read_file('./City Data/Makati_point.geojson')

manila_transpo_gdf = manila_points_gdf[manila_points_gdf['amenity'] == 'transportation']
mandaluyong_transpo_gdf = mandaluyong_points_gdf[mandaluyong_points_gdf['amenity'] == 'transportation']
makati_transpo_gdf = makati_points_gdf[makati_points_gdf['amenity'] == 'transportation']

In [ ]:
# Index for optimization
manila_idx = rtree_index.Index()
for i, row in manila_transpo_gdf.iterrows():
    manila_idx.insert(i, row['geometry'].bounds)
    
mandaluyong_idx = rtree_index.Index()
for i, row in mandaluyong_transpo_gdf.iterrows():
    mandaluyong_idx.insert(i, row['geometry'].bounds)
    
makati_idx = rtree_index.Index()
for i, row in makati_transpo_gdf.iterrows():
    makati_idx.insert(i, row['geometry'].bounds)
    

In [ ]:
# Function to Extract all the stop points per row
def extract_points(df):
    routes = []
    columns = list(df.columns)
    
    for index, row in df.iterrows():
        points = []
        for col in columns:
            cell_value = row[col]
            if pd.isna(cell_value):
                continue
                
            if cell_value.endswith(']'):
                cell_value = cell_value[:-1] + ',' # Remove the last bracket if it's there
                
            data = eval(cell_value)[0]
            
            lat = data[0]
            lon = data[1]
            points.append((lat, lon))
        routes.append(points)
    return routes

In [ ]:
# Reading the original jeepney route data
manila_df = pd.read_csv('Original Jeepney Routes/Original Data/Manila Routes.csv')
makati_df = pd.read_csv('Original Jeepney Routes/Original Data/Makati Routes.csv')
mandaluyong_df = pd.read_csv('Original Jeepney Routes/Original Data/Mandaluyong Routes.csv')

manila_route_stops = extract_points(manila_df)
makati_route_stops = extract_points(makati_df)
mandaluyong_route_stops = extract_points(mandaluyong_df)

### Stop Placement

In [ ]:
manila_excluded_stops = [[14.634778170776876, 120.99271635348593], [14.634709797650444, 120.9926832475171], [14.634687527719784, 120.99270234878283], [14.632037849596644, 120.99179926456718], [14.63201915056513, 120.99180163899968], [14.63199332654228, 120.99178690311312], [14.629815787147514, 120.9910864368258], [14.629760253875283, 120.99108208862654], [14.629754205373912, 120.99105684985572], [14.627542415902836, 120.99029841737115], [14.627513571798906, 120.99031178550912], [14.627334908304396, 120.99026121356665], [14.625754423040169, 120.99102843767942], [14.6383688, 120.9809056697085], [14.6348973197085, 120.9810326697085], [14.635359271701084, 120.96210659268327], [14.63533, 120.96213], [14.6083736197085, 120.9615864197085], [14.6083322697085, 120.9607424], [14.6087639197085, 120.9651623197085], [14.5989322, 120.964597], [14.5608934, 120.9883491], [14.5590401, 120.9869151], [14.5618191, 120.9974419], [14.56158879999997, 120.9982477], [14.5751095197085, 121.0088462697085], [14.5750185, 121.0073474], [14.5861209697085, 121.0170452697085], [14.6012427697085, 121.0196449697085], [14.617365381201216, 121.00124069680852], [14.6209582197085, 120.9964309197085], [14.625754423040169, 120.99102843767942]]

In [ ]:
# Checks if point is out of the specific geographic location
def remove_if_out_of_bounds(city_graph, routes, tolerance):
    nodes = ox.graph_to_gdfs(city_graph, edges=False)
    min_x, min_y, max_x, max_y = nodes.total_bounds
    
    new_routes = []
    for route in routes:
        new_route = []
        for point in route:
            lat = point[0]
            long = point[1]
            
            if min_x <= long <= max_x and min_y <= lat <= max_y:
                # Convert the graph to GeoDataFrame for easier spatial operations
                nearest_edge, distance_to_edge = ox.distance.nearest_edges(city_graph, X=long, Y=lat, return_dist=True)
                
            
                # Return True if any distance is less than the tolerance
                if distance_to_edge < tolerance and [lat, long] not in manila_excluded_stops:
                    new_route.append(point)
                else:
                    break
        new_routes.append(new_route)
            
    return new_routes

In [ ]:
# This function checks if there are stops that are transpo stop and labels them as such
def get_close_points(idx, point, max_distance=50):
    close_points = [i for i in idx.intersection((point.x - max_distance, point.y - max_distance, point.x + max_distance, point.y + max_distance))]
    return close_points

def check_transpo_stops(routes, point_gdf, city_idx):
    transpo_points = []
    non_transpo_points = []
    new_route_list_with_stop_id = []
    
    stop_id = 0
    for route in routes:
        new_route = []
        for point in route:
            lat = point[0]
            long = point[1]
            isTranspo = False
            
            close_points = get_close_points(city_idx, Point(long, lat))
            for i, row in point_gdf.loc[close_points].iterrows():
                distance_to_point = geodesic(point, (row['y'], row['x'])).meters
                
                # Check if distance is within 100 meters and it's a transportation amenity
                if distance_to_point <= 50 and row['amenity'] == 'transportation':
                    isTranspo = True
                    break
                
            new_stop = stopCandidate(lat, long, isTranspo, stop_id)
            new_route.append(new_stop)
            
            if isTranspo:
                transpo_points.append(new_stop)
            else:
                non_transpo_points.append(new_stop)
            stop_id += 1
        
        new_route_list_with_stop_id.append(new_route)
        
    return transpo_points, non_transpo_points, new_route_list_with_stop_id
        
        

In [ ]:
# x - lon - 120
# y - lat - 14
# LineString(lon, lat)
# geodesic(lat, lon)

def add_point_to_graph(point, graph_of_stops, list_of_stops, is_transpo):
    lat = point.lat
    long = point.long
    # Find the nearest edge to the location point
    nearest_edge = ox.distance.nearest_edges(graph_of_stops, X=long, Y=lat, return_dist=False)
    node1, node2, key = nearest_edge
    edge = graph_of_stops.get_edge_data(node1, node2)
    data = list(edge.values())[0]
        
    # get the line
    if 'geometry' not in data:
        gdf_edges = ox.graph_to_gdfs(graph_of_stops, nodes=False, edges=True)
        line = gdf_edges.loc[nearest_edge]['geometry']
    else:
        line = data['geometry']
        
    point_a = (graph_of_stops.nodes[node1]['y'], graph_of_stops.nodes[node1]['x'])
    point_c = (graph_of_stops.nodes[node2]['y'], graph_of_stops.nodes[node2]['x']) 
    

    graph_of_stops.add_node(point.id, x=long, y=lat, isTranspo=is_transpo)
    list_of_stops.append(point)

    # Get the points
    point_b = (lat, long) # y,x
    
    # Calculate the new distances
    distance_ab = geodesic(point_a, point_b).meters
    distance_bc = geodesic(point_b, point_c).meters
        
    # Get the edge data and adjust distances
    edge_data = data.copy()
    edge_data['length'] = distance_ab
    graph_of_stops.add_edge(node1, point.id, **edge_data)
    
    edge_data['length'] = distance_bc
    graph_of_stops.add_edge(point.id, node2, **edge_data)

    # Remove the original edge
    graph_of_stops.remove_edge(node1, node2)
        
def calculate_coordinate_along_edge(edge, position):
    # Calculate the coordinate along the edge at the given position
    point = edge.interpolate(position)
    return point.x, point.y

In [ ]:
# CREATING STOPS
# It should return a list of coordinates/nodes for stop and a graph of stops
# if residential area, check if the population density

def place_stops_on_roads(transpo_points, non_transpo_points, graph_of_stops, list_of_stops):
    
    for point in transpo_points:
        add_point_to_graph(point, graph_of_stops, list_of_stops, True)
        
    for point in non_transpo_points:
        add_point_to_graph(point, graph_of_stops, list_of_stops, False)

In [ ]:
def merge_stops(stops_list):
    stop_dict = {}
    # Sort the stops by latitude and then by longitude
    stops = sorted(stops_list, key=lambda x: (x.lat, x.long), reverse=True)
    

    grouped_stops = []
    current_group = [stops[0]]

    for i in range(1, len(stops)):
        distance = geodesic((current_group[-1].lat, current_group[-1].long), (stops[i].lat, stops[i].long)).meters
        if distance <= 20:
            current_group.append(stops[i])
        else:
            grouped_stops.append(current_group)
            current_group = [stops[i]]

    # Last group
    if current_group:
        grouped_stops.append(current_group)

    
    for group in grouped_stops:
        
        if len(group) == 1:
            continue
        
        # Getting the main stop
        main_stop = None
        for stop in group:
            if stop.isTranspo:
                main_stop = stop
                break
        # If no transpo stop, let it be the first stop
        if main_stop == None:
            main_stop = group[0]
        
        for stop in group:
            if stop.id != main_stop.id:
                stop_dict[stop.id] = main_stop
        
    return stop_dict

In [ ]:
def change_merged_stops(route_list, non_transpo_points_Manila, transpo_points_Manila, stop_dict):
    transpo_list = []
    non_transpo_list = []
    for route in route_list:
        for stop in route:
            if stop.id in stop_dict:
                index = route.index(stop)
                route[index] = stop_dict[stop.id]
    
    # Make sure there are no duplicates
    for i, route in enumerate(route_list):
        seen = set()
        unique_stops = []
        for stop in route:
            if stop.id not in seen:
                unique_stops.append(stop)
                seen.add(stop.id)
        route_list[i] = unique_stops
                
    for stop in non_transpo_points_Manila:
        if stop.id not in stop_dict:
            non_transpo_list.append(stop)
            
    for stop in transpo_points_Manila:
        if stop.id not in stop_dict:
            transpo_list.append(stop)
            
    return transpo_list, non_transpo_list

### Route Network Generation

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    # Use geopy's geodesic function to calculate the distance
    distance = geodesic((lat1, lon1), (lat2, lon2)).kilometers
    return distance

In [ ]:
def generate_route_network(routes, graph_of_stops):
    overall_graph = nx.MultiDiGraph() # The route network graph
    route_network = []
    reverse_route_network = []
    num_routes = 0 # Count number of routes
    
    # Check first if there are paths
    for route in routes:
        prev_stop = None
        for stop in route:
            if prev_stop == None:
                prev_stop = stop
                continue
            
            if nx.has_path(graph_of_stops, prev_stop.id, stop.id):
                print(f"{prev_stop.id} - {stop.id}")
                prev_stop = stop
            else:
                route_index = routes.index(route)
                stop_index = route.index(prev_stop)
                routes[route_index] = route[:stop_index+1]
                break
        print()
    
    new_routes = []
    for route in routes:
        if len(route) > 3:
            new_routes.append(route)
    routes = new_routes
    
    print()
    print("Real route")
    for route in routes:
        route_id = f'{num_routes}-A' # This will be used as a key for the edge
        route_list = []
        prev_stop = None
        totalDistance = 0
        
        for stop in route: 
            if prev_stop == None:
                prev_stop = stop
                continue

            print(f"{prev_stop.id} - {stop.id}")
            
            path = nx.shortest_path(graph_of_stops, prev_stop.id, stop.id)
            route_list.append(path)
            
            distance_travelled = 0
            # Get the total distance from point A to point B
            for i in range(len(path)-1):
                node_data = graph_of_stops.nodes[path[i]]
                next_node_data = graph_of_stops.nodes[path[i+1]]
                distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
            
            overall_graph.add_node(prev_stop.id, lat=prev_stop.lat, lon=prev_stop.long, isTranspo=prev_stop.isTranspo) # The origin
            overall_graph.add_node(stop.id, lat=stop.lat, lon=stop.long, isTranspo=stop.isTranspo) # The dest
            overall_graph.add_edge(prev_stop.id, stop.id, key=route_id, road_path = path, distance = distance_travelled) # Add edge
            prev_stop = stop
            totalDistance += distance_travelled
            
            
        print()
        new_route_obj = Route(route_list, route_id, totalDistance)
        reverse_route = get_reverse_route(new_route_obj, graph_of_stops, overall_graph)
        route_network.append(new_route_obj)
        reverse_route_network.append(reverse_route)
        num_routes += 1
               
    return route_network, overall_graph, reverse_route_network
        
        

In [ ]:
# Method 1
# Iterate through each node in reverse and get the shortest path

def get_reverse_route(route, graph_of_stops, overall_graph):
    paths = route.route
    reverse_paths = paths[::-1]
    reverse_route_list = []
    reverse_route_id = route.route_id[:-1] + 'B' # Reverse Route ID
    totalDistance = 0
    
    stop_list = []
    for connection in reverse_paths:
        reverse_route_path = connection[::-1]
        end_node = reverse_route_path[-1]
        
        # Getting the edges of the start node only for the first connection
        if reverse_paths.index(connection) == 0:
            start_node = reverse_route_path[0] # Start node is critical as it should be the same as the last stop previous connection (If this is not the first connection)
                    
        if not nx.has_path(graph_of_stops, start_node, end_node):
            return None
        else:
            stop_list.append([start_node, end_node])
                
        start_node = end_node # Next connection start node is the end node of the previous connection
        
    print(stop_list)
    for connection in stop_list:
        start_node = connection[0]
        end_node = connection[-1]
        
        reverse_route = nx.shortest_path(graph_of_stops, start_node, end_node, weight='length')
        reverse_route_list.append(reverse_route)
        
        distance_travelled = 0
        # Get the total distance from point A to point B
        for i in range(len(reverse_route)-1):
            node_data = graph_of_stops.nodes[reverse_route[i]]
            next_node_data = graph_of_stops.nodes[reverse_route[i+1]]
            distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
        
        # Finally, Add the edge
        if reverse_route[0] not in overall_graph:
            overall_graph.add_node(reverse_route[0], **graph_of_stops.nodes[reverse_route[0]])
        if reverse_route[-1] not in overall_graph:
            overall_graph.add_node(reverse_route[-1], **graph_of_stops.nodes[reverse_route[-1]])
            
        overall_graph.add_edge(reverse_route[0], reverse_route[-1], key=reverse_route_id, road_path = reverse_route, distance = distance_travelled) # Add edge
        totalDistance += distance_travelled

    # print("REVERSE ROUTE DISTANCE: ", totalDistance)
    return Route(reverse_route_list, reverse_route_id, totalDistance)

### Fitness Function

In [ ]:
# Fitness function

def compute_fitness_score(road_snapped_network_graph, num_failure_removal,
                          weight_random_failure, weight_network_coverage, weight_targeted_failure, weight_connectivity, total_road_network_distance):

    random_failure_robustness = compute_random_failure_robustness(road_snapped_network_graph, num_failure_removal)
    weighted_random_failure_robustness = weight_random_failure * random_failure_robustness

    targeted_failure_robustness = compute_targeted_failure_robustness(road_snapped_network_graph, num_failure_removal)
    weighted_targeted_failure_robustness = weight_targeted_failure * targeted_failure_robustness

    connectivity_score, transpo_stop_ratio,num_intersections, avg_transpo_degree = compute_connectivity(road_snapped_network_graph)
    weighted_connectivity = weight_connectivity * connectivity_score
    
    network_coverage = get_network_coverage(road_snapped_network_graph, total_road_network_distance)
    weighted_network_coverage = weight_network_coverage * network_coverage
    
    data = {
        'Random Failure Score': weighted_random_failure_robustness,
        'Target Failure Score': weighted_targeted_failure_robustness,
        'Connectivity': weighted_connectivity,
        'Transpo Stop Ratio': transpo_stop_ratio,
        'Num Intersections': num_intersections,
        'Average Transpo Degree': avg_transpo_degree,
        'Network Coverage': weighted_network_coverage
    }

    # Will use this return for now to utilize target and random failure nodes 
    return weighted_connectivity + weighted_network_coverage - weighted_random_failure_robustness - weighted_targeted_failure_robustness, data
    # return weighted_radius_of_gyration


### WRITTEN IN PSEUDOCODE
def compute_connectivity(network):
    # External connectivity - measure how connected is the jeepney route network with other modes of transpo
    
    # Get the ratio of transportation stops to total stops in the network
    transpo_stops = [node for node, node_data in network.nodes(data=True) if node_data['isTranspo'] == True]
    total_stops = len(network.nodes(data=True))
    transpo_stop_ratio = len(transpo_stops) / total_stops

    # Get the average degree of all transportation stops in the network
    if len(transpo_stops) > 0:
        avg_transpo_degree = sum(network.degree(stop) for stop in transpo_stops) / len(transpo_stops)
    else:
        avg_transpo_degree = 1

    # Find a way to normalize the two values and combine them 

    # Internal connectivity - measure how connected is each jeepney route to other jeepney routes
                    
    # This counts how many nodes have intersections (Meaning node is connected to more than one route by route ID)
    num_intersections = 0
    for node, node_data in network.nodes(data=True):
        unique_ids = set()  # Use a set to automatically handle uniqueness
        edges = network.edges(node, keys=True)  # Get all edges connected to the node with keys
        for u, v, key in edges:
            if isinstance(key, str) and '-' in key:
                id_str = key.split('-')[0]  # Get route id number
                unique_ids.add(id_str)  # Add to the set
        if len(unique_ids) > 1:
            num_intersections += 1

    
    # Change these weights based on what the expected values for 
    # the transpo_stop_ratio, avg_transpo_degree, and num_intersections will be
    external_weight = 0.5
    internal_weight = 0.5
    
    # TODO: Delete this
    # print("Transpo stop ratio: ", transpo_stop_ratio)
    # print("Num intersections: ", num_intersections)
    # print("Average degree: ", avg_transpo_degree)

    # Formula subject to change
    return external_weight * (transpo_stop_ratio * avg_transpo_degree) + internal_weight * num_intersections, transpo_stop_ratio,num_intersections, avg_transpo_degree


def compute_random_failure_robustness(road_snapped_network_graph, num_removals):
    graph_copy = road_snapped_network_graph.copy() # Make a copy
    
    for i in range(num_removals):
        selected_node = random.choice(list(graph_copy.nodes()))
        graph_copy.remove_node(selected_node)

    diameter, avg_path_length = compute_network_statistics(graph_copy)
    return compute_failure_robustness(graph_copy, diameter)

def compute_targeted_failure_robustness(road_snapped_network_graph, num_removals):
    graph_copy = road_snapped_network_graph.copy() # Make a copy
    
    for i in range(num_removals):
        node_degrees = graph_copy.degree()
        # Iterate over the DegreeView object to find the maximum degree
        max_degree = max(degree for _, degree in node_degrees)
        max_degree_node = get_node_with_degree(node_degrees, max_degree)
        graph_copy.remove_node(max_degree_node)

    diameter, avg_path_length = compute_network_statistics(graph_copy)
    return compute_failure_robustness(graph_copy, diameter)

def compute_failure_robustness(road_snapped_network_graph, max_path_length):
    return float(max_path_length) / float(len(road_snapped_network_graph) - 1)

def compute_network_statistics(road_snapped_network_graph):
    path_lengths = get_path_lengths(road_snapped_network_graph) # Get the sum of all possible
    avg_path_length = np.mean(path_lengths)
    max_path_length = max(path_lengths)

    #network_size = len(path_lengths)
    #gcc = sorted(nx.connected_component_subgraphs(road_snapped_network_graph), key=len, reverse=True)
    #giant_component_fraction = float(float(gcc[0].order()) / float(network_size))
    #return max_path_length, avg_path_length, giant_component_fraction
    return max_path_length, avg_path_length

def get_node_with_degree(node_degrees, degree):
    # Iterate over the DegreeView object to find the node with the specified degree
    for node, _ in node_degrees:
        if _ == degree:
            return node
    return None  # Return None if no node with the specified degree is found

def get_path_lengths(snapped_road_network_graph):
    return [sum(nx.single_source_shortest_path_length(snapped_road_network_graph, n).values())
            for n in snapped_road_network_graph]
    
def get_network_coverage(network_graph, total_road_network_distance):
    
    travelled_distance = 0
    edge_set = [] # This will contain all the edges travelled
    
    for node1, node2, edge_data in network_graph.edges(data=True):
        string_key1 = f"{node1}_{node2}"
        string_key2 = f"{node2}_{node1}"
        
        if string_key1 not in edge_set and string_key2 not in edge_set:
            edge_set.append(string_key1)
            edge_set.append(string_key2)
            travelled_distance += edge_data['distance'] * 1000 # km to meters   
    return travelled_distance / total_road_network_distance

### Network Analysis Metrics

In [ ]:
# Longest route, shortest route, average route length, network diamater

### Visualizations

In [ ]:
def add_markers(routes, map):
    added = set()
    for route in routes:
        for stop in route:
            if stop.id not in added:
                #popup_text = f"Name: {stop.name}<br>Type: {stop.a_type}<br>Coordinates: {stop.getLat()}, {stop.getLong()}"
                lat = stop.lat
                long = stop.long
                
                if stop.isTranspo:
                    folium.Marker(location=[lat, long],icon=folium.Icon(color='green')).add_to(map)
                else:
                    folium.Marker(location=[lat, long],icon=folium.Icon(color='blue')).add_to(map)
                added.add(stop.id)

In [ ]:
# This is to visualize the stops
def plot_stops_on_map(stops):
    # Iterate over the nodes in the network
    map_center = (14.599512, 120.984222)
    network_map = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
    for stop in stops:
        
        if stop.isTranspo:
            marker_color = 'blue'
        else:
            marker_color = 'red'
            
        folium.Marker(location=[stop.lat, stop.long], popup=f"Transportation: {stop.isTranspo}", icon=folium.Icon(color=marker_color)).add_to(network_map)
        
    return network_map

In [ ]:
# Test Manila Map

# Create a map centered at the initial location
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
for route in manila_route_stops:
    for stop in route:
        folium.Marker(location=[stop[0], stop[1]], popup=f"{stop[0]}, {stop[1]}", icon=folium.Icon(color='blue')).add_to(m)
        
m.save('test_manila.html')

In [ ]:
# Test Makati Map

# Create a map centered at the initial location
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
for route in makati_route_stops:
    for stop in route:
        folium.Marker(location=[stop[0], stop[1]], popup=f"{stop[0]}, {stop[1]}", icon=folium.Icon(color='blue')).add_to(m)
        
m.save('test_makati.html')

In [ ]:
# Test Mandaluyong Map

# Create a map centered at the initial location
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
for route in mandaluyong_route_stops:
    for stop in route:
        folium.Marker(location=[stop[0], stop[1]], popup=f"{stop[0]}, {stop[1]}", icon=folium.Icon(color='blue')).add_to(m)
        
m.save('test_mandaluyong.html')

## Main

### Creating the Stops

In [ ]:
# File Paths
pikl_filepath = "Original Jeepney Routes/Saved Networks/"
map_filepath = "Original Jeepney Routes/Maps/"

#### Manila Stops

In [ ]:
# MANILA
# Remove all points that are out of bounds
tolerance = 0.0009
manila_route_stops_new = remove_if_out_of_bounds(MANILA_GRAPH, manila_route_stops, tolerance)

# Remove all routes that are less than 3 stops
temp_array = [] # For storing routes that are above 3 stops
for route in manila_route_stops_new:
    if len(route) > 3:
        temp_array.append(route)
manila_route_stops_new = temp_array

# Get transpo stops
transpo_points_Manila, non_transpo_points_Manila, complete_route_list_Manila = check_transpo_stops(manila_route_stops_new, manila_points_gdf, manila_idx)


In [ ]:
merged_list = transpo_points_Manila + non_transpo_points_Manila
stop_dict = merge_stops(merged_list)

route_list_Manila = copy.deepcopy(complete_route_list_Manila)
transpo_points_Manila, non_transpo_points_Manila = change_merged_stops(route_list_Manila, non_transpo_points_Manila, transpo_points_Manila, stop_dict)

In [ ]:
# MANILA STOPCANDIDATES
graph_of_stops_Manila = copy.deepcopy(MANILA_GRAPH)
list_of_stops_Manila = []

place_stops_on_roads(transpo_points_Manila, non_transpo_points_Manila, graph_of_stops_Manila, list_of_stops_Manila)

# Visualize the stops
map = plot_stops_on_map(list_of_stops_Manila)
map.save("stop_map_manila.html")

In [ ]:
#Export stops to pickle
file_path = f'{pikl_filepath}stop_list_Manila.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(list_of_stops_Manila, f)
    
file_path = f'{pikl_filepath}stop_graph_Manila.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(graph_of_stops_Manila, f)

#### Makati

In [ ]:
# MAKATI
# Remove all points that are out of bounds
tolerance = 0.009
makati_route_stops_new = remove_if_out_of_bounds(MAKATI_GRAPH, makati_route_stops, tolerance)

# Remove all routes that are less than 3 stops
temp_array = [] # For storing routes that are above 3 stops
for route in makati_route_stops_new:
    if len(route) > 3:
        temp_array.append(route)
makati_route_stops_new = temp_array

# Get transpo stops
transpo_points_Makati, non_transpo_points_Makati, complete_route_list_Makati = check_transpo_stops(makati_route_stops_new, makati_points_gdf, makati_idx)


In [ ]:
merged_list = transpo_points_Makati + non_transpo_points_Makati
stop_dict = merge_stops(merged_list)

route_list_Makati = copy.deepcopy(complete_route_list_Makati)
transpo_points_Makati, non_transpo_points_Makati = change_merged_stops(route_list_Makati, non_transpo_points_Makati, transpo_points_Makati, stop_dict)

In [ ]:
# MAKATI STOPCANDIDATES
graph_of_stops_Makati = copy.deepcopy(MAKATI_GRAPH)
list_of_stops_Makati = []

place_stops_on_roads(transpo_points_Makati, non_transpo_points_Makati, graph_of_stops_Makati, list_of_stops_Makati)

# Visualize the stops
map = plot_stops_on_map(list_of_stops_Makati)
map.save("stop_map_makati.html")

In [ ]:
#Export stops to pickle
file_path = f'{pikl_filepath}stop_list_Makati.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(list_of_stops_Makati, f)
    
file_path = f'{pikl_filepath}stop_graph_Makati.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(graph_of_stops_Makati, f)

#### Mandaluyong

In [ ]:
# MANDALUYONG
# Remove all points that are out of bounds
tolerance = 0.009
mandaluyong_route_stops_new = remove_if_out_of_bounds(MANDALUYONG_GRAPH, mandaluyong_route_stops, tolerance)

# Remove all routes that are less than 3 stops
temp_array = [] # For storing routes that are above 3 stops
for route in mandaluyong_route_stops_new:
    if len(route) > 3:
        temp_array.append(route)
mandaluyong_route_stops_new = temp_array

# Get transpo stops
transpo_points_Mandaluyong, non_transpo_points_Mandaluyong, complete_route_list_Mandaluyong = check_transpo_stops(mandaluyong_route_stops_new, mandaluyong_points_gdf, mandaluyong_idx)


In [ ]:
merged_list = transpo_points_Mandaluyong + non_transpo_points_Mandaluyong
stop_dict = merge_stops(merged_list)

route_list_Mandaluyong = copy.deepcopy(complete_route_list_Mandaluyong)
transpo_points_Mandaluyong, non_transpo_points_Mandaluyong = change_merged_stops(route_list_Mandaluyong, non_transpo_points_Mandaluyong, transpo_points_Mandaluyong, stop_dict)

In [ ]:
# MANDALUYONG STOPCANDIDATES
graph_of_stops_Mandaluyong = copy.deepcopy(MANDALUYONG_GRAPH)
list_of_stops_Mandaluyong = []

place_stops_on_roads(transpo_points_Mandaluyong, non_transpo_points_Mandaluyong, graph_of_stops_Mandaluyong, list_of_stops_Mandaluyong)

# Visualize the stops
map = plot_stops_on_map(list_of_stops_Mandaluyong)
map.save("stop_map_mandaluyong.html")

In [ ]:
#Export stops to pickle
file_path = f'{pikl_filepath}stop_list_Mandaluyong.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(list_of_stops_Mandaluyong, f)
    
file_path = f'{pikl_filepath}stop_graph_Mandaluyong.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(graph_of_stops_Mandaluyong, f)

### Creating the routes

In [ ]:
# Manila
route_network_Manila, route_network_graph_Manila, reverse_route_network_Manila = generate_route_network(route_list_Manila, graph_of_stops_Manila)
Manila_Route_Network = networkObj(route_network_Manila, route_network_graph_Manila)

In [ ]:
# MAKATI ROUTES
route_network_Makati, route_network_graph_Makati, reverse_route_network_Makati = generate_route_network(route_list_Makati, graph_of_stops_Makati)
Makati_Route_Network = networkObj(route_network_Makati, route_network_graph_Makati)

In [ ]:
# MANDALUYONG
route_network_Mandaluyong, route_network_graph_Mandaluyong, reverse_route_network_Mandaluyong = generate_route_network(route_list_Mandaluyong, graph_of_stops_Mandaluyong)
Mandaluyong_Route_Network = networkObj(route_network_Mandaluyong, route_network_graph_Mandaluyong)

In [ ]:
# Creating Maps for visualization

# MANILA
map_center = (14.599512, 120.984222)
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Plotting in the Map
add_markers(route_list_Manila, m)
    
for route in Manila_Route_Network.route_network:
    for connection in route.route:
        ox.plot_route_folium(graph_of_stops_Manila, connection, route_map=m, tiles='openstreetmap', route_color="green")

m.save(f"{map_filepath}Manila Route Map.html")

# MAKATI
map_center = (14.599512, 120.984222)
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Plotting in the Map
add_markers(route_list_Makati, m)
    
for route in Makati_Route_Network.route_network:
    for connection in route.route:
        ox.plot_route_folium(graph_of_stops_Makati, connection, route_map=m, tiles='openstreetmap', route_color="green")

m.save(f"{map_filepath}Makati Route Map.html")

# MANDALUYONG
map_center = (14.599512, 120.984222)
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Plotting in the Map
add_markers(route_list_Mandaluyong, m)
    
for route in Mandaluyong_Route_Network.route_network:
    for connection in route.route:
        ox.plot_route_folium(graph_of_stops_Mandaluyong, connection, route_map=m, tiles='openstreetmap', route_color="green")

m.save(f"{map_filepath}Mandaluyong Route Map.html")

In [ ]:
len(Mandaluyong_Route_Network.route_network)

### Fitness Function Computation

In [ ]:
# Evaluate the fitness of each network in the population
num_failure_removal = 4
weight_random_failure = 0.15
weight_targeted_failure = 0.15
weight_connectivity = 0.5
weight_network_coverage = 0.2

Manila_fitness_score, data_Manila = compute_fitness_score(Manila_Route_Network.graph, num_failure_removal, weight_random_failure, weight_network_coverage, weight_targeted_failure, weight_connectivity, TOTAL_NETWORK_ROAD_DISTANCE_MANILA)
Manila_Route_Network.fitness_score = Manila_fitness_score
print(f"Manila Route Network Score {Manila_Route_Network.fitness_score}")
print(f"Weighted Connectivity: {data_Manila['Connectivity']}")
print(f"- Transpo Stop Ratio: {data_Manila['Transpo Stop Ratio']}")
print(f"- Num Intersections: {data_Manila['Num Intersections']}")
print(f"- Average Transpo Stop Degree: {data_Manila['Average Transpo Degree']}")
print(f"Weighted Network Coverage: {data_Manila['Network Coverage']}")
print(f"Weighted Random Failure Score: {data_Manila['Random Failure Score']}")
print(f"Weighted Target Failure Score: {data_Manila['Target Failure Score']}")
print()

Makati_fitness_score, data_Makati = compute_fitness_score(Makati_Route_Network.graph, num_failure_removal, weight_random_failure, weight_network_coverage, weight_targeted_failure, weight_connectivity, TOTAL_NETWORK_ROAD_DISTANCE_MAKATI)
Makati_Route_Network.fitness_score = Makati_fitness_score
print(f"Makati Route Network Score {Makati_Route_Network.fitness_score}")
print(f"Weighted Connectivity: {data_Makati['Connectivity']}")
print(f"- Transpo Stop Ratio: {data_Makati['Transpo Stop Ratio']}")
print(f"- Num Intersections: {data_Makati['Num Intersections']}")
print(f"- Average Transpo Stop Degree: {data_Makati['Average Transpo Degree']}")
print(f"Weighted Network Coverage: {data_Makati['Network Coverage']}")
print(f"Weighted Random Failure Score: {data_Makati['Random Failure Score']}")
print(f"Weighted Target Failure Score: {data_Makati['Target Failure Score']}")
print()

Mandaluyong_fitness_score, data_Mandaluyong = compute_fitness_score(Mandaluyong_Route_Network.graph, num_failure_removal, weight_random_failure, weight_network_coverage, weight_targeted_failure, weight_connectivity, TOTAL_NETWORK_ROAD_DISTANCE_MANDALUYONG)
Mandaluyong_Route_Network.fitness_score = Mandaluyong_fitness_score
print(f"Mandaluyong Route Network Score {Mandaluyong_Route_Network.fitness_score}")
print(f"Weighted Connectivity: {data_Mandaluyong['Connectivity']}")
print(f"- Transpo Stop Ratio: {data_Mandaluyong['Transpo Stop Ratio']}")
print(f"- Num Intersections: {data_Mandaluyong['Num Intersections']}")
print(f"- Average Transpo Stop Degree: {data_Mandaluyong['Average Transpo Degree']}")
print(f"Weighted Network Coverage: {data_Mandaluyong['Network Coverage']}")
print(f"Weighted Random Failure Score: {data_Mandaluyong['Random Failure Score']}")
print(f"Weighted Target Failure Score: {data_Mandaluyong['Target Failure Score']}")
print()


In [ ]:
Manila_fitness_score, data_Manila = compute_fitness_score(Manila_Route_Network.graph, num_failure_removal, weight_random_failure, weight_network_coverage, weight_targeted_failure, weight_connectivity, TOTAL_NETWORK_ROAD_DISTANCE_MANILA)
Manila_Route_Network.fitness_score = Manila_fitness_score
print(f"Manila Route Network Score {Manila_Route_Network.fitness_score}")
print(f"Weighted Connectivity: {data_Manila['Connectivity']}")
print(f"- Transpo Stop Ratio: {data_Manila['Transpo Stop Ratio']}")
print(f"- Num Intersections: {data_Manila['Num Intersections']}")
print(f"- Average Transpo Stop Degree: {data_Manila['Average Transpo Degree']}")
print(f"Weighted Network Coverage: {data_Manila['Network Coverage']}")
print(f"Weighted Random Failure Score: {data_Manila['Random Failure Score']}")
print(f"Weighted Target Failure Score: {data_Manila['Target Failure Score']}")
print()

In [ ]:
Makati_fitness_score, data_Makati = compute_fitness_score(Makati_Route_Network.graph, num_failure_removal, weight_random_failure, weight_network_coverage, weight_targeted_failure, weight_connectivity, TOTAL_NETWORK_ROAD_DISTANCE_MAKATI)
Makati_Route_Network.fitness_score = Makati_fitness_score
print(f"Makati Route Network Score {Makati_Route_Network.fitness_score}")
print(f"Weighted Connectivity: {data_Makati['Connectivity']}")
print(f"Weighted Network Coverage: {data_Makati['Network Coverage']}")
print(f"Weighted Random Failure Score: {data_Makati['Random Failure Score']}")
print(f"Weighted Target Failure Score: {data_Makati['Target Failure Score']}")
print()